In [ ]:
# Proje: New York City Metro Verileri İle Keşifçi Veri Analizi
# Veri Kaynağı: web.mta.info/developers/turnstile.html
# Veri Tarih Aralığı: 2022 Mart - 2022 Mayıs

 ### Proje Hakkında

WomenTechWomenYes(WTWY) derneği, her yıl yaz başında düzenlediği galaya katılımı arttırmak için NYC metrosunun istasyon girişlerine saha ekipleri yerleştiriyor. Yerleştirilen ekipler aracılığıyla, ulaştıkları kişilerin e-posta adreslerini toplamakta ve kayıt olan kişilere ücretsiz bilet göndermektedir.

Bu galaya katılım sağlayabilecek ve derneğin destek olabilecek çok sayıda kişiye ulaşmaları için saha ekiplerinin yerleşimlerini optimize etmemiz isteniyor.

### Kullanılan Veri Seti

Bu projede, MTA Ne York City turnikelerinden elde edilen ve 2022 Mart-Mayıs tarihleri arasını kapsayan giriş ve çıkış verileri kullanılmıştır.

Veriler haftalık olarak üretilmekte ve her 4 saatte bir, istasyon, turnike, tarih ve saat bazında giriş ve çıkış bilgilerinin kümülatif toplamlarından oluşmaktadır.

Çalışma yaptığımız tarih aralığındaki veri seti 2.739.787 satır, 11 kolondan oluşmaktadır.

### Kullanılan Araçlar

Bu çalışmada Jupyter notebook üzerinde Python, Pandas, Matplotlib, NumPy ve Seaborn kullanılmıştır.

### Çalışmanın Amacı ve Çözüm Yaklaşımları

En yoğun kullanıma sahip metro istasyonlarının tespit edilmesi ve saha elemanlarının bu noktalara yerleştirilmesiyle çok fazla sayıda kullanıcıya ulaşma hedeflenmektedir. Bu nedenle;

- En yoğun kullanılan istasyonların tespiti
- Yoğun olan istasyonların en kalabalık olduğu saatlerin tespiti
- Her bir istasyonun en kabalalık olduğu gün ve saatler
- Teknoloji şirketlerine yakın olan metro istasyonlarının tespiti

Problemin çözümü için geliştireceğimiz çözümler olacaktır.

Field Description

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station. Normally lines are represented by one character. LINENAME 456NQR represents train server for 4, 5, 6, N, Q, and R trains.
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 
ENTRIES  = The cumulative entry register value for a device
EXIST    = The cumulative exit register value for a device

In [13]:
# Kütüphanelerin yüklenmesi
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# 05/03/2022 tarihi başlangıç olacak şekilde 13 haftalık verinin çekilmesi
start_date = pd.Timestamp("Saturday, March 05, 2022 00:00:00")
date = start_date
url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"

source = []

for i in range(13):
    file_name = str(date.year)[2:4] + str(date.month).zfill(2) + str(date.day).zfill(2)
    date += pd.Timedelta(days=7)
    file_name = url.format(file_name)
    source.append(pd.read_csv(file_name))

In [15]:
# Çekilen verilerin bir dataframe olarak kaydedilmesi ve görüntülenmesi
dataframe = pd.concat(source)
dataframe.info() # Dataframe içeriğinin görüntülenmesi
dataframe.shape # Dataframe satır ve sütun sayıları

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2739787 entries, 0 to 211706
Data columns (total 11 columns):
 #   Column                                                                Dtype 
---  ------                                                                ----- 
 0   C/A                                                                   object
 1   UNIT                                                                  object
 2   SCP                                                                   object
 3   STATION                                                               object
 4   LINENAME                                                              object
 5   DIVISION                                                              object
 6   DATE                                                                  object
 7   TIME                                                                  object
 8   DESC                                                           

(2739787, 11)

In [16]:
dataframe.isnull().sum() # Dataframe içerisinde boş hücre kontrolü

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

In [17]:
dataframe.columns # Dataframe kolon isimlerini gösterme

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

### Sütun isimlerini kontrol ettiğimizde EXITS sütununda boşlukların olduğu görülüyor. Çalışmamızda sorun yaşanmaması için isim değişikliğini gerçekleştiriyoruz.
.
.
.


In [18]:
dataframe.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace = True)



### Çalışmamızda Linename ve Division sütunlarına ihtiyacımız olmayacağı için veri setinden çıkarıyoruz.
.
.
.

In [27]:
dataframe.drop(["LINENAME","DIVISION"], axis=1,inplace = True)

KeyError: "['LINENAME', 'DIVISION'] not found in axis"

In [28]:
dataframe.head() #İlk 5 hücre

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,02/26/2022,03:00:00,REGULAR,7689737,2671257
1,A002,R051,02-00-00,59 ST,02/26/2022,07:00:00,REGULAR,7689741,2671278
2,A002,R051,02-00-00,59 ST,02/26/2022,11:00:00,REGULAR,7689758,2671380
3,A002,R051,02-00-00,59 ST,02/26/2022,15:00:00,REGULAR,7689791,2671465
4,A002,R051,02-00-00,59 ST,02/26/2022,19:00:00,REGULAR,7689858,2671530


In [29]:
dataframe.isnull().sum() #Boş hücre kontrolü

C/A        0
UNIT       0
SCP        0
STATION    0
DATE       0
TIME       0
DESC       0
ENTRIES    0
EXITS      0
dtype: int64

In [30]:
dataframe.tail() #Son 5 hücre

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS
211702,TRAM2,R469,00-05-01,RIT-ROOSEVELT,05/27/2022,05:00:00,REGULAR,0,382
211703,TRAM2,R469,00-05-01,RIT-ROOSEVELT,05/27/2022,09:00:00,REGULAR,0,382
211704,TRAM2,R469,00-05-01,RIT-ROOSEVELT,05/27/2022,13:00:00,REGULAR,0,383
211705,TRAM2,R469,00-05-01,RIT-ROOSEVELT,05/27/2022,17:00:00,REGULAR,0,386
211706,TRAM2,R469,00-05-01,RIT-ROOSEVELT,05/27/2022,21:00:00,REGULAR,0,388


Veri setimizde tarih ve saatlerin ayrı sütunlarda verildiğini görüyoruz.

- Date ve Time sütunlarını birleştirerek datetime adından yeni bir sütun ekliyoruz.
- En kalabalık gün tespiti için veri setimize Gün adında bir sütun ekliyoruz.

In [42]:
dataframe['DATETIME'] = dataframe['DATE'] + " " + dataframe['TIME']
dataframe['DATETIME'] = pd.to_datetime(dataframe['DATETIME'],format='%m/%d/%Y %H:%M:%S') #Datetime adlı sütunumuz.
dataframe['DAY'] = dataframe['DATETIME'].dt.day_name() #Day adlı sütunumuz.

In [43]:
dataframe.head()

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY
0,A002,R051,02-00-00,59 ST,02/26/2022,03:00:00,REGULAR,7689737,2671257,2022-02-26 03:00:00,Saturday
1,A002,R051,02-00-00,59 ST,02/26/2022,07:00:00,REGULAR,7689741,2671278,2022-02-26 07:00:00,Saturday
2,A002,R051,02-00-00,59 ST,02/26/2022,11:00:00,REGULAR,7689758,2671380,2022-02-26 11:00:00,Saturday
3,A002,R051,02-00-00,59 ST,02/26/2022,15:00:00,REGULAR,7689791,2671465,2022-02-26 15:00:00,Saturday
4,A002,R051,02-00-00,59 ST,02/26/2022,19:00:00,REGULAR,7689858,2671530,2022-02-26 19:00:00,Saturday


.
.
### Turnikeler için Station, Control Area, Unit ve SCP verileriyle ayrı bir sütun oluşturuyoruz.
.
.
.


In [46]:
dataframe['TURNSTILE'] = dataframe['STATION'] + "_" + dataframe['C/A'] + "_" + dataframe['UNIT'] + "_" + dataframe['SCP']

In [56]:
dataframe.head()

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,TURNSTILE
0,A002,R051,02-00-00,59 ST,02/26/2022,03:00:00,REGULAR,7689737,2671257,2022-02-26 03:00:00,Saturday,59 ST_A002_R051_02-00-00
1,A002,R051,02-00-00,59 ST,02/26/2022,07:00:00,REGULAR,7689741,2671278,2022-02-26 07:00:00,Saturday,59 ST_A002_R051_02-00-00
2,A002,R051,02-00-00,59 ST,02/26/2022,11:00:00,REGULAR,7689758,2671380,2022-02-26 11:00:00,Saturday,59 ST_A002_R051_02-00-00
3,A002,R051,02-00-00,59 ST,02/26/2022,15:00:00,REGULAR,7689791,2671465,2022-02-26 15:00:00,Saturday,59 ST_A002_R051_02-00-00
4,A002,R051,02-00-00,59 ST,02/26/2022,19:00:00,REGULAR,7689858,2671530,2022-02-26 19:00:00,Saturday,59 ST_A002_R051_02-00-00


In [65]:
dataframe.sort_values(by=['DATETIME'])
dataframe.head()

,C/A,UNIT,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY,TURNSTILE
0,A002,R051,02-00-00,59 ST,02/26/2022,03:00:00,REGULAR,7689737,2671257,2022-02-26 03:00:00,Saturday,59 ST_A002_R051_02-00-00
1,A002,R051,02-00-00,59 ST,02/26/2022,07:00:00,REGULAR,7689741,2671278,2022-02-26 07:00:00,Saturday,59 ST_A002_R051_02-00-00
2,A002,R051,02-00-00,59 ST,02/26/2022,11:00:00,REGULAR,7689758,2671380,2022-02-26 11:00:00,Saturday,59 ST_A002_R051_02-00-00
3,A002,R051,02-00-00,59 ST,02/26/2022,15:00:00,REGULAR,7689791,2671465,2022-02-26 15:00:00,Saturday,59 ST_A002_R051_02-00-00
4,A002,R051,02-00-00,59 ST,02/26/2022,19:00:00,REGULAR,7689858,2671530,2022-02-26 19:00:00,Saturday,59 ST_A002_R051_02-00-00


In [66]:
turnstiles[['prev_date', 'prev_entry']] = turnstiles.groupby(['C/A', 'UNIT', 'SCP', 'STATION'])['DATE,ENTRIES'].shift(1)
turnstiles.dropna(subset=[prev_date], axis=0)
turnstiles[turnstiles['ENTRIES']<turnstiles['prev_entry']].head()

def get_daily_counts(dataframe, max_counter=1500000):
    counter = dataframe['ENTRIES'] - dataframe['ENTRIES']
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        return 0
    return counter

KeyError: 'Column not found: DATE,ENTRIES'

In [ ]:
turnstiles['daily_entries']=turnstilesdaily.apply(get_daily_counts, axis=1, max_counter=40000)